# What is this about?

This notebooks deals with getting models pretrained and using them in the forecasting. It has two steps:
1. Make sure that the current model is working
2. Make Predictions 

In [2]:
def GetPreviousModel(asset_name):

    import boto3
    from datetime import datetime
    
    s3 = boto3.resource(
            service_name='s3',
            aws_access_key_id='QyvycO9kc2cm58K8',
            aws_secret_access_key='tKtUrdQzQgWfhfBwhbQF3yGbyZ43oPn92iGAT7g0',
            endpoint_url='https://s3.tebi.io'
        )

    
    my_bucket = s3.Bucket('test-pf')
    list_objects = []
    for my_bucket_object in my_bucket.objects.all():
        if asset_name in my_bucket_object.key and "model" in  my_bucket_object.key:
            list_objects.append(
                {
                    "model_name": my_bucket_object.key,
                    "last_modified": datetime.strftime(my_bucket_object.last_modified, "%Y-%m-%d %H:%M:%S")
                })
    
    if len(list_objects) == 0:
        raise ValueError("No model was found for this asset_name")
    else:
        return list_objects

In [3]:
models = GetPreviousModel("de-nieuwe-dokken-pv-017A-xxxxx9A1")

In [4]:
models

[{'model_name': 'model_Ghent_pv_de-nieuwe-dokken-pv-017A-xxxxx9A1.json',
  'last_modified': '2023-05-15 04:08:27'}]

In [8]:
def ObtainProphetModel(models):
    from prophet.serialize import model_from_json
    import boto3
    s3_client = boto3.client('s3',
            aws_access_key_id='QyvycO9kc2cm58K8',
            aws_secret_access_key='tKtUrdQzQgWfhfBwhbQF3yGbyZ43oPn92iGAT7g0',
            endpoint_url='https://s3.tebi.io'
        )
    dir_model ="/Users/fablab/Downloads" + models[0]["model_name"]
    model_name = models[0]["model_name"]

    with open(dir_model, 'wb') as f:
        s3_client.download_fileobj('test-pf', model_name, f)

    with open(dir_model, "r") as fin:
        m = model_from_json(fin.read())
    return m

In [9]:
m = ObtainProphetModel(models)

/Users/fablab/opt/anaconda3/lib/python3.8/site-packages/prophet/serialize.py:136: FutureWarning: The behavior of Timestamp.utcfromtimestamp is deprecated, in a future version will return a timezone-aware Timestamp with UTC timezone. To keep the old behavior, use Timestamp.utcfromtimestamp(ts).tz_localize(None). To get the future behavior, use Timestamp.fromtimestamp(ts, 'UTC')
  setattr(model, attribute, pd.Timestamp.utcfromtimestamp(attr_dict[attribute]))


In [28]:
def model_predict(model, num_days: int = 1, freq: str = "1H"):
    from datetime import datetime
    today = datetime.today().date()
    last_date_model = model.history.ds.max().date()
    difference = today - last_date_model
    
    dias_a_predecir = difference.days + num_days
    
    future = model.make_future_dataframe(periods=dias_a_predecir*24, freq=freq, include_history=False)
    predicciones = model.predict(future)[["ds", "yhat"]]
    
    return predicciones [ predicciones.ds >= datetime.combine(datetime.today().date(), datetime.min.time()) ]

In [30]:
model_predict(m, num_days = 4)

/Users/fablab/opt/anaconda3/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/Users/fablab/opt/anaconda3/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)


,ds,yhat
24,2023-05-15 00:00:00,282.613496
25,2023-05-15 01:00:00,-79.269186
26,2023-05-15 02:00:00,-206.389265
27,2023-05-15 03:00:00,218.027368
28,2023-05-15 04:00:00,929.659310
...,...,...
139,2023-05-19 19:00:00,322.959869
140,2023-05-19 20:00:00,-194.930405
141,2023-05-19 21:00:00,-210.714593
142,2023-05-19 22:00:00,218.894208
